In [1]:
%pip install langchain_chroma
%pip install langchain_openai
%pip install langchain_google_genai
%pip install langchain_huggingface
%pip install langchain_community
%pip install FlagEmbedding
!pip install python-dotenv
%pip install pypdf

  Using cached langchain_huggingface-0.2.0-py3-none-any.whl.metadata (941 bytes)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached langchain_huggingface-0.2.0-py3-none-any.whl (27 kB)
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl (207.5 MB)
  Attempting uninstall: nvidia-cusparse-cu12
    Found existing installation: nvidia-cusparse-cu12 12.5.1.3
    Uninsta

In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.docstore.document import Document

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the files and index them in a vector database


In [5]:
docs= []
for doc in PyPDFDirectoryLoader("/content/data").load():


    metadata ={
        "source": doc.metadata["source"],
        "page": doc.metadata["page"],

    }
    doc_text = ' '.join(doc.page_content.split())
    # Remove newlines and extra spaces
    docs.append(Document(page_content=doc_text,
                                   metadata=metadata))

# PyPDFDirectoryLoader loads all PDFs in a directory

In [25]:
# prompt: import from utils.py create_chroma_db

import utils

In [26]:
utils.find_best_embedding

AttributeError: module 'utils' has no attribute 'find_best_embedding'

In [13]:
utils.create_chroma_db

<function utils.create_chroma_db(docs, embedding_list, embedding_names, similarity_type='cosine')>

In [153]:
from pprint import pprint
pprint(docs[1])

Document(metadata={'source': '/content/data/instructgpt.pdf', 'page': 1}, page_content='1.3B 6B 175B Model size 0.2 0.4 0.6Win rate against SFT 175B Model PPO-ptx PPO SFT GPT (prompted) GPT Figure 1: Human evaluations of various models on our API prompt distribution, evaluated by how often outputs from each model were preferred to those from the 175B SFT model. Our InstructGPT models (PPO-ptx) as well as its variant trained without pretraining mix (PPO) signiﬁcantly outperform the GPT-3 baselines (GPT, GPT prompted); outputs from our 1.3B PPO-ptx model are preferred to those from the 175B GPT-3. Error bars throughout the paper are 95% conﬁdence intervals. used for many recent large LMs—predicting the next token on a webpage from the internet—is different from the objective “follow the user’s instructions helpfully and safely” (Radford et al., 2019; Brown et al., 2020; Fedus et al., 2021; Rae et al., 2021; Thoppilan et al., 2022). Thus, we say that the language modeling objective is mis

In [154]:
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
from langchain_chroma import Chroma
from FlagEmbedding import BGEM3FlagModel
# The splitting and chunking strategy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv(".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACE_TOKEN')

In [155]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [156]:
# Download the embeddings
open_ai_large = OpenAIEmbeddings(model="text-embedding-3-large")
open_ai_small = OpenAIEmbeddings(model="text-embedding-3-small")


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


In [157]:
def create_chroma_db(docs, embedding_list, embedding_names, similarity_type="cosine"):
  splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  chunked_docs = splitter.split_documents(docs)

  for embedding, name in zip(embedding_list, embedding_names):
    for similarity in similarity_type:
      persist_dir = f"./capstone_db_{name}_{similarity}"
      chromba_db = Chroma.from_documents(documents=chunked_docs,
                                        collection_name=f'capstone_db_{name}_{similarity}',
                                        embedding=embedding,
                                        collection_metadata={"hnsw:space": similarity},
                                         persist_directory=persist_dir)

      print(f"Chroma DB created for embedding: {name} with similarity: {similarity} in directory: {persist_dir}")

In [158]:
docs = docs
embedding_list = [open_ai_large, open_ai_small, hf]
embedding_names = ["open_ai_large", "open_ai_small","mp_allnet"]
similarity_type = ["cosine", "l2"]

In [159]:
create_chroma_db(docs=docs, embedding_list=embedding_list, embedding_names=embedding_names, similarity_type=similarity_type)

Chroma DB created for embedding: open_ai_large with similarity: cosine in directory: ./capstone_db_open_ai_large_cosine
Chroma DB created for embedding: open_ai_large with similarity: l2 in directory: ./capstone_db_open_ai_large_l2
Chroma DB created for embedding: open_ai_small with similarity: cosine in directory: ./capstone_db_open_ai_small_cosine
Chroma DB created for embedding: open_ai_small with similarity: l2 in directory: ./capstone_db_open_ai_small_l2
Chroma DB created for embedding: mp_allnet with similarity: cosine in directory: ./capstone_db_mp_allnet_cosine
Chroma DB created for embedding: mp_allnet with similarity: l2 in directory: ./capstone_db_mp_allnet_l2


### From the above embeddings, we can infer that text-embedding-3-large retrieves relevant data in the top.

### change to euclidean embeddings

In [160]:

def find_best_embedding(embedding_list, embedding_names,question):
  # load from disk
  embedding_name =[]
  retriever_details = []
  similarity_type = []
  combined_list_euc = {}

  for embedding, name in zip(embedding_list, embedding_names):
    for similarity  in similarity_type:
      chroma_db = Chroma(persist_directory=f"./capstone_db_{name}_{similarity}",
                        collection_name=f'capstone_db_{name}_{similarity}',
                         embedding_function=embedding)
      similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                search_kwargs={"k": 5, "score_threshold": 0.0})
      embedding_name.append(name)

      embed_search = similarity_retriever.invoke(f"{question}", k=5)
      retriever_details.append(embed_search)
      similarity_type = similarity_type.append(similarity)
      details = dict(zip(embedding_name, retriever_details,similarity))
      combined_list_euc.update(details)

  return combined_list_euc

In [161]:
question="what is attention layer"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names, question=question)

In [162]:
print(output)

{}


In [137]:
for embedding, retriever, similarity in output.items():
    print(f"Embedding: {embedding}")
    print(f"Similarity_search: {similarity}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}")
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

## Cosine similarity is better than Euclidean similarity.  We find the repetition of documents

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point. also show the top 3 context documents of the answer.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
       |
    prompt_template
      |
    chatgpt
)

In [ ]:
from IPython.display import display, Markdown
query = "What is gemini"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))


Gemini is a family of highly capable multimodal models that includes evaluations at both the model and product levels, focusing on safety mitigations and user experience. It addresses critical policy areas such as hate speech and dangerous content, and adopts a user-centric approach to maximize diversity across various topics and user journeys.

Top 3 context documents:
1. Gemini: A Family of Highly Capable Multimodal Models 7.4.2. Gemini Advanced
2. Gemini: A Family of Highly Capable Multimodal Models 7.4.2. Gemini Advanced
3. Gemini: A Family of Highly Capable Multimodal Models 7.4.2. Gemini Advanced